# The Parcels Kernel loop


This tutorial explains how Parcels loops through Kernels, and what happens under the hood when you combine Kernels. 

This is probably not very relevant when you only use the built-in Advection kernels, but can be important when you writing and combining adding your Kernels!

When you run a Parcels simulation (i.e. a call to `pset.execute()`), the Kernel loop is the main part of the code that is executed. It is the part of the code that loops through all particles and executes the Kernels that are defined for each particle.

In order to make sure that the movements of a particle in the different Kernels can be summed, all Kernels add to a _change_ in position. This is important, because there are situations where movement kernels would otherwise not commute. Take the example of advecting particles by currents _and_ winds. If the particle would first be moved by the currents and then by the winds, the result could be different from first moving by the winds and then by the currents. By adding the changes in position, the order of the Kernels does not matter anymore.

## Basic implementation

Below shows a structured overview of the Kernel loop is implemented. Note that this is for longitude only, but the same is done for latitude and depth.

1. Define an extra variable `particle.lon_nextloop` for each particle, which is the longitude at the end of the Kernel loop. Inititalise it to `particle.lon`.
2. Also define an extra variable `particle.time_nextloop` for each particle, which is the time at the end of the Kernel loop. Inititalise it to `particle.time`.
3. Within the loop, for each particle:<br>
    1. Update `particle.lon` with `particle.lon_nextloop`<br>
    2. Update `particle.time` with `particle.time_nextloop`<br>
    3. Set local variable `particle_dlon = 0`<br>
    4. For each Kernel in the list of Kernels:<br>
        1. Execute the Kernel<br>
        2. Update `particle_dlon` with the change in longitude, if needed<br>
    5. Update `particle.lon_nextloop` with `particle.lon + particle_dlon`<br>
    6. Update `particle.time_nextloop` with `particle.time + particle.dt`<br>
    7. If `outputdt` is a multiple of `particle.time`, write `particle.lon` and `particle.time` to zarr output file<br>

The main advantage of this implementation is that, when using Field Sampling with e.g. `particle.temp = fieldset.Temp[particle.time, particle.depth, particle.lat, particle.lon]`, the particle location stays the same throughout the entire Kernel loop. This also ensure that in the output file, the particle location is the same as the location of the sampled field.

## Caveats

There are a few important considerations to take into account when writing Kernels

### Do not update particle locations directly in Kernels
Try to avoid updating `particle.lon` directly in a Kernel, as it can interfere with the loop above. This will also throw a warning. 

Instead, update the local variable `particle_dlon`.

### Be careful with updating particle variables that do not depend on Fields.
While assigning the interpolated value of a `Field` to a Particle goes well in the loop above, this is not necessarily so for assigning other attributes. For example, a line like `particle.age += particle.dt` is executed directly so may result in the age being `dt` at `time = 0` in the output file. 

A workaround is to either initialise the age to `-dt`, or to increase the `age` only when `particle.time > 0` (using an `if` statement).


### The last time is not written to file
Because the location at the start of the loop is written at the end of the Kernel loop, the last time of the particle is not written to file. This is similar behaviour to e.g. `np.arange(start, stop)`, which also doesn't include the `stop` value itself. 

If you do want to write the last time to file, you can increase the `runtime` or `endtime` by `dt` (although this may cause a TimeExtrapolationError if your run was to the end of the available hydrodynamic data), or you can call `pfile.write_latest_locations(pset, time=endtime)`. Note that in the latter case, the particle locations (longitude, latitude and depth) will be updated, but other variables will _not_ be updated as the Kernels are not run again.

## Working with Status Codes